In [19]:
from pyspark import SparkConf, SparkContext
from re import split

# Create SparkContext object
conf = SparkConf().setMaster("local").setAppName("RatingsValueCounts")
sc = SparkContext(conf=conf)

# Create lines RDD from data file
data_file_path = "../data/customer-orders.csv"
lines = sc.textFile(data_file_path)
lines.take(2)

['44,8602,37.19', '35,5368,65.89']

line = ('customer_id', 'product_id', 'product_prize')


In [20]:
def parse_line(line: str) -> tuple[str, float]:
    row = line.split(",")
    customer_id = row[0]
    amount_spent = float(row[2])
    return (customer_id, amount_spent)


customer_expenses = lines.map(parse_line)
customer_expenses.take(3)

[('44', 37.19), ('35', 65.89), ('2', 40.64)]

In [21]:
total_spent_per_customer = (
    customer_expenses.reduceByKey(lambda x, y: x + y)  # sum of values for each key
    .mapValues(lambda x: round(x, 2))  # round values to 2 decimals
    .sortBy(lambda x: x[1], ascending=False)  # sort by value
)

total_spent_per_customer.take(3)

[('68', 6375.45), ('73', 6206.2), ('39', 6193.11)]

In [22]:
sc.stop()